In [105]:
import pandas as pd
import numpy as np

child_mortality = pd.read_csv('child-mortality.csv')
child_mortality.head(100)

,Entity,Code,Year,"Child mortality (Select Gapminder, v10) (2017)"
0,Afghanistan,AFG,1957,381.194
1,Afghanistan,AFG,1958,375.190
2,Afghanistan,AFG,1959,369.361
3,Afghanistan,AFG,1960,363.700
4,Afghanistan,AFG,1961,357.500
...,...,...,...,...
95,Albania,ALB,2013,14.900
96,Albania,ALB,2014,14.400
97,Albania,ALB,2015,14.000
98,Albania,ALB,2016,13.500


In [106]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [107]:
child_mortality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13512 entries, 0 to 13511
Data columns (total 4 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Entity                                          13512 non-null  object 
 1   Code                                            13512 non-null  object 
 2   Year                                            13512 non-null  int64  
 3   Child mortality (Select Gapminder, v10) (2017)  13512 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 422.4+ KB


### TASK 1 : Minimum values of child mortality in different countries

In [108]:
child_mortality.rename(columns={"Child mortality (Select Gapminder, v10) (2017)": "Child_mortality"}, inplace=True)
df = child_mortality[["Entity", "Code", "Child_mortality"]]
df.groupby("Code").min().sort_values("Child_mortality")

,Entity,Child_mortality
Code,,
ISL,Iceland,2.100
HKG,Hong Kong,2.282
FIN,Finland,2.300
SVN,Slovenia,2.300
LUX,Luxembourg,2.400
...,...,...
MLI,Mali,110.600
SLE,Sierra Leone,113.500
CAF,Central African Republic,123.600


### TASK 2 : Least value of CM in countries in 2016

In [109]:
# Check if 2017 is available for all countries
df = child_mortality[child_mortality["Year"] == 2016]
df = df[["Entity", "Code", "Child_mortality"]]  # Use only relevant columns
df.sort_values("Child_mortality", inplace=True)

df.head()

,Entity,Code,Child_mortality
5528,Iceland,ISL,2.100
5217,Hong Kong,HKG,2.282
10883,Slovenia,SVN,2.300
4087,Finland,FIN,2.300
7229,Luxembourg,LUX,2.400


### TASK 3 : Plot this on the map. Create a Choropleth

In [110]:
import folium
from folium import Choropleth
import geopandas as gpd

In [174]:
worldfilepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(worldfilepath)
print(len(world))
world.head()

177


,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [112]:
child_mortality.rename(columns={"Code": "iso_a3"}, inplace=True)
child_mortality.head()

,Entity,iso_a3,Year,Child_mortality
0,Afghanistan,AFG,1957,381.194
1,Afghanistan,AFG,1958,375.190
2,Afghanistan,AFG,1959,369.361
3,Afghanistan,AFG,1960,363.700
4,Afghanistan,AFG,1961,357.500


In [113]:
df = child_mortality[child_mortality["Year"] == 2016]
df = df.merge(world, on="iso_a3")

c_m = df[["iso_a3", "Child_mortality"]].set_index("iso_a3")
country_geometry = df[["iso_a3", "geometry"]].set_index("iso_a3")

country_geom_gdf = gpd.GeoDataFrame(country_geometry, crs="EPSG:4326")
country_geom_gdf.head()

,geometry
iso_a3,
AFG,"POLYGON ((66.51861 37.36278, 67.07578 37.35614..."
ALB,"POLYGON ((21.02004 40.84273, 20.99999 40.58000..."
DZA,"POLYGON ((-8.68440 27.39574, -8.66512 27.58948..."
AGO,"MULTIPOLYGON (((12.99552 -4.78110, 12.63161 -4..."
ARG,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000..."


In [114]:
# Finally, lets create the map
m = folium.Map(location=[0.0, 0.0], tiles='openstreetmap', zoom_start=2)

Choropleth(geo_data=country_geom_gdf.__geo_interface__,
          data=c_m["Child_mortality"], key_on='feature.id', fill_color='YlGnBu', legend_name='Child Mortality(2016)').add_to(m)

embed_map(m, 'Child_mortality-2016.html')

### TASK 4 : Compare this to how it was back in 1990

In [115]:
# Get data for 1990
data_1990 = child_mortality[child_mortality["Year"] == 1990].copy()

# Merge data to make sure only countries for which data is available are plotted
merged_df = data_1990.merge(world, on="iso_a3")

# Convert to GeoDataFrame
geom_1990 = gpd.GeoDataFrame(merged_df[["iso_a3", "geometry"]].set_index("iso_a3"), crs="EPSG:4326")

# 1990's data for child_mortality
data = merged_df[["iso_a3", "Child_mortality"]].set_index("iso_a3")

In [116]:
m = folium.Map(location=[0.0, 0.0], tiles='openstreetmap', zoom_start=2)
Choropleth(geo_data=geom_1990.__geo_interface__, data=data["Child_mortality"],
          key_on='feature.id', fill_color='YlGnBu', legend_name="Child Mortality(1990)").add_to(m)

embed_map(m, 'Child_mortality-1990.html')

#### Overall, the two maps look almost the same. Child mortality rate had reduced drastically since the 1990's(as you can see in the legend)

### TASK 5 : Make a similar map with the plot corrsponding to the percentage of reduction in the child mortality rate. Use the first occurence and last occurence of data for each country 

In [117]:
# First merge the two dataframes
merged = child_mortality.merge(world, on="iso_a3")

In [118]:
# child_mortality.groupby("iso_a3").apply()
reduction = merged.groupby('iso_a3')[['Child_mortality', 'Year']].agg(['first', 'last'])

# Find the reduction in child mortality
reduction["Percentage"] = (reduction['Child_mortality']["last"]
                           - reduction['Child_mortality']['first'])* -100.0 / reduction['Child_mortality']['first']
reduction["Year_diff"] = reduction["Year"]["last"] - reduction['Year']['first']

reduction['Percentage_per_year'] = reduction['Percentage'] / reduction['Year_diff']

print(len(reduction))
reduction.head()

167


Child_mortality        Year       Percentage Year_diff  \
                 first  last first  last                        
iso_a3                                                          
AFG            381.194  70.4  1957  2016  81.531714        59   
AGO            236.400  82.5  1980  2016  65.101523        36   
ALB             90.800  13.5  1978  2016  85.132159        38   
ARE            204.200   7.7  1960  2016  96.229187        56   
ARG            238.820  11.1  1911  2016  95.352148       105   

       Percentage_per_year  
                            
iso_a3                      
AFG               1.381893  
AGO               1.808376  
ALB               2.240320  
ARE               1.718378  
ARG               0.908116

#### Now plot the percentage on the choropleth. This would tell us how the rates of child mortality have dropped across different regions.
As data is available from different start points, some of the countries might not be representative of the actual drop they have experienced. Eg. Russia - dat available from 2012, where the child mortality is already quite small. So, we will plot the percentage reduction per year for which data is available

In [176]:
# Create the GeoDataFrame
geometry_data = merged[merged["Year"] == 2016]
geometry_data = geometry_data[["iso_a3", "geometry"]].set_index("iso_a3")

geometry_data_gdf = gpd.GeoDataFrame(geometry_data, crs="EPSG:4326")

print(len(geometry_data_gdf))
geometry_data_gdf.head()

167


,geometry
iso_a3,
AFG,"POLYGON ((66.51861 37.36278, 67.07578 37.35614..."
ALB,"POLYGON ((21.02004 40.84273, 20.99999 40.58000..."
DZA,"POLYGON ((-8.68440 27.39574, -8.66512 27.58948..."
AGO,"MULTIPOLYGON (((12.99552 -4.78110, 12.63161 -4..."
ARG,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000..."


In [120]:
# Finally, create the base map and the Choropleth
m = folium.Map(location=[0.0, 0.0], tiles='openstreetmap', zoom_start=2)

# Choropleth
Choropleth(geo_data=geometry_data_gdf.__geo_interface__, data=reduction['Percentage'],
          key_on='feature.id', fill_color='YlGnBu', legend_name="Percentage reduction in Child Mortality").add_to(m)

m_1 = folium.Map(location=[0.0, 0.0], tiles='openstreetmap', zoom_start=2)

# Choropleth
Choropleth(geo_data=geometry_data_gdf.__geo_interface__, data=reduction['Percentage_per_year'],
          key_on='feature.id', fill_color='YlGnBu', legend_name="Percentage reduction in Child Mortality").add_to(m_1)

embed_map(m, "Child_mortality_percentage_reduction.html")

In [121]:
embed_map(m_1, 'Child_mortality_percentage_reduction_per_year.html')

### Lets create a time slider choropleth which plots the transition of child mortality over the years

In [122]:
# creating 11 bins to check values of child_mortality
bins = np.linspace(min(child_mortality['Child_mortality']), max(child_mortality['Child_mortality']), 11)
bins

array([  2.1  ,  70.098, 138.096, 206.094, 274.092, 342.09 , 410.088,
       478.086, 546.084, 614.082, 682.08 ])

In [123]:
child_mortality['color'] = pd.cut(
    child_mortality['Child_mortality'], bins,
    labels=['#FFEBEB','#F8D2D4','#F2B9BE','#EBA1A8','#E58892','#DE6F7C','#D85766','#D13E50','#CB253A','#C50D24'],
    include_lowest=False)
child_mortality['color'].replace(np.nan, '#32CD32', inplace=True)
child_mortality.color.value_counts()

#FFEBEB    6480
#F8D2D4    2619
#F2B9BE    1766
#EBA1A8    1371
#E58892     827
#DE6F7C     303
#D85766      97
#D13E50      38
#C50D24       5
#CB253A       5
Name: color, dtype: int64

In [124]:
plot_data = child_mortality[["iso_a3", "Year", "color"]]
plot_data.head()

,iso_a3,Year,color
0,AFG,1957,#DE6F7C
1,AFG,1958,#DE6F7C
2,AFG,1959,#DE6F7C
3,AFG,1960,#DE6F7C
4,AFG,1961,#DE6F7C


In [125]:
# Now add rows for countries which do not have recordings for a particular year
for year in plot_data['Year'].unique():
    diff = set(plot_data['iso_a3'].unique()) - set(plot_data[plot_data['Year'] == year]['iso_a3'])
    for country in diff:
        plot_data = pd.concat([plot_data, pd.DataFrame([[country, year, '#0073CF']], columns=['iso_a3', 'Year', 'color'])],
                              ignore_index=True)
plot_data.sort_values('Year', inplace=True)
plot_data.head()

,iso_a3,Year,color
11491,SWE,1800,#DE6F7C
44661,MYT,1800,#0073CF
44662,AGO,1800,#0073CF
44663,CAF,1800,#0073CF
44664,AUT,1800,#0073CF


In [177]:
p_data = plot_data.copy()
# p_data = p_data[(p_data['Year'] > 1799)]
p_data['Year'] = pd.to_datetime(p_data.Year, format='%Y')

p_data['Year'] = (p_data['Year'].astype(np.int64) // 10**9)

plot_dict = {}
for i in p_data['iso_a3'].unique():
    if i in geometry_data.index:
        plot_dict[i] = {}
        for j in p_data[p_data['iso_a3'] == i].set_index(['iso_a3']).values:
            plot_dict[i][j[0]] = {'color': j[1], 'opacity': 0.7 }

len(list(plot_dict.items()))

167

In [183]:
# Merge the two data frames to get rows which are available in both
# geometry_data = plot_data[plot_data['Year'] == 2016].merge(geometry_data, on='iso_a3')
# geometry_data = geometry_data[["iso_a3", "geometry"]].set_index("iso_a3")

geometry_gdf = gpd.GeoDataFrame(geometry_data, crs='EPSG:4326')


# print(len(geometry_data))
# geometry_data.head()

In [186]:
from folium.plugins import TimeSliderChoropleth
# geometry_data = geometry_data[geometry_data.index == 'IND']

m = folium.Map(location=[0.0, 0.0], tiles='openstreetmap', zoom_start=2)
time_slider = TimeSliderChoropleth(geometry_gdf.to_json(),
                                  styledict=plot_dict).add_to(m)
embed_map(m, 'time_slider_choropleth.html')

In [29]:
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read Expenditure data
expenditure = pd.read_csv('data/healthcare-expenditure-vs-gdp.csv')
expenditure.head()

,Entity,Code,Year,"Total population (Gapminder, HYDE & UN)",Continent,"GDP per capita, PPP (constant 2011 international $)","Health expenditure per capita, PPP (constant 2011 international $)"
0,Abkhazia,OWID_ABK,2015,NaN,Asia,NaN,NaN
1,Afghanistan,AFG,1800,3280000.0,NaN,NaN,NaN
2,Afghanistan,AFG,1801,3280000.0,NaN,NaN,NaN
3,Afghanistan,AFG,1802,3280000.0,NaN,NaN,NaN
4,Afghanistan,AFG,1803,3280000.0,NaN,NaN,NaN


In [9]:
# Clean up data
expenditure.rename(columns={'Total population (Gapminder, HYDE & UN)': 'Population',
                            'GDP per capita, PPP (constant 2011 international $)': 'GDP',
                            'Health expenditure per capita, PPP (constant 2011 international $)': 'Expenditure'}, inplace=True)
expenditure = expenditure[(expenditure['GDP'].notna() & expenditure['Expenditure'].notna())]
print(expenditure.info())
expenditure.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4525 entries, 203 to 48166
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Entity       4525 non-null   object 
 1   Code         3635 non-null   object 
 2   Year         4525 non-null   int64  
 3   Population   3655 non-null   float64
 4   Continent    0 non-null      object 
 5   GDP          4525 non-null   float64
 6   Expenditure  4525 non-null   float64
 7   Percentage   4525 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 318.2+ KB
None


,Entity,Code,Year,Population,Continent,GDP,Expenditure,Percentage
203,Afghanistan,AFG,2002,22601000.0,NaN,1063.635574,75.983514,7.143754
204,Afghanistan,AFG,2003,23681000.0,NaN,1099.194507,89.440178,8.136884
205,Afghanistan,AFG,2004,24727000.0,NaN,1062.249360,87.520903,8.239205
206,Afghanistan,AFG,2005,25654000.0,NaN,1136.123214,88.323062,7.774074
207,Afghanistan,AFG,2006,26433000.0,NaN,1161.124889,86.856016,7.480334


In [10]:
# Find counts of Year. This would tell us which year has all the required data available
expenditure.Year.value_counts()

2011    231
2010    230
2008    230
2012    230
2009    230
2007    230
2013    230
2014    230
2006    229
2005    229
2003    229
2004    229
2002    228
2001    227
2000    226
1999    220
1998    219
1997    218
1996    216
1995    214
Name: Year, dtype: int64

### Lets find percentage of gdp spent as health expenditure

In [11]:
expenditure['Percentage'] = expenditure['Expenditure'] * 100.0 / expenditure['GDP']
expenditure.head()

,Entity,Code,Year,Population,Continent,GDP,Expenditure,Percentage
203,Afghanistan,AFG,2002,22601000.0,NaN,1063.635574,75.983514,7.143754
204,Afghanistan,AFG,2003,23681000.0,NaN,1099.194507,89.440178,8.136884
205,Afghanistan,AFG,2004,24727000.0,NaN,1062.249360,87.520903,8.239205
206,Afghanistan,AFG,2005,25654000.0,NaN,1136.123214,88.323062,7.774074
207,Afghanistan,AFG,2006,26433000.0,NaN,1161.124889,86.856016,7.480334


### TASK 1 : Get all countries that spend less than 5% of their GDP on healthcare in 2011

In [17]:
filtered_df = expenditure[(expenditure['Year'] == 2011) & (expenditure['Percentage'] < 5.0)].copy()
filtered_df['Entity'].unique()

array(['Angola', 'Arab World', 'Armenia', 'Azerbaijan', 'Bahrain',
       'Bangladesh', 'Belarus', 'Benin', 'Bhutan', 'Brunei', 'Cameroon',
       'Cape Verde', 'Central African Republic', 'Chad', 'China', 'Congo',
       'Democratic Republic of Congo', 'Dominican Republic',
       'Early-demographic dividend', 'East Asia & Pacific (IDA & IBRD)',
       'East Asia & Pacific (excluding high income)', 'Equatorial Guinea',
       'Eritrea', 'Fiji', 'Fragile and conflict affected situations',
       'Gabon', 'Ghana', 'Guinea', 'IDA blend', 'IDA only', 'IDA total',
       'India', 'Indonesia', 'Iraq', 'Kazakhstan', 'Kuwait', 'Laos',
       'Latvia', 'Least developed countries: UN classification',
       'Lower middle income', 'Madagascar', 'Malaysia', 'Mauritania',
       'Mauritius', 'Middle East & North Africa', 'Mongolia', 'Myanmar',
       'Nigeria', 'Oman', 'Other small states', 'Pakistan',
       'Papua New Guinea', 'Peru', 'Philippines',
       'Pre-demographic dividend', 'Qatar',
  

In [19]:
# Plot all of these countries on the map
worldfilepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(worldfilepath)
world.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [47]:
filtered_df.rename(columns={'Code': 'iso_a3'}, inplace=True)
five_percent = filtered_df.merge(world, on='iso_a3')
five_percent.head()

,Entity,iso_a3,Year,Population,Continent,GDP,Expenditure,Percentage,pop_est,continent,name,gdp_md_est,geometry
0,Angola,AGO,2011,24221000.0,NaN,5911.254092,220.500282,3.730178,29310273,Africa,Angola,189000.0,"MULTIPOLYGON (((12.99552 -4.78110, 12.63161 -4..."
1,Armenia,ARM,2011,2877000.0,NaN,7022.103943,252.650423,3.597931,3045191,Asia,Armenia,26300.0,"POLYGON ((46.50572 38.77061, 46.14362 38.74120..."
2,Azerbaijan,AZE,2011,9146000.0,NaN,15754.152363,784.521610,4.979777,9961396,Asia,Azerbaijan,167900.0,"MULTIPOLYGON (((46.40495 41.86068, 46.68607 41..."
3,Bangladesh,BGD,2011,149272992.0,NaN,2570.850145,81.400564,3.166290,157826578,Asia,Bangladesh,628400.0,"POLYGON ((92.67272 22.04124, 92.65226 21.32405..."
4,Belarus,BLR,2011,9415000.0,NaN,17166.695643,816.230045,4.754730,9549747,Europe,Belarus,165400.0,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."


In [48]:
# Create the Geodataframe for the plot
five_percent = five_percent[["iso_a3", "Percentage", 'geometry', "Entity"]]
data = five_percent[['iso_a3', 'Percentage']].set_index('iso_a3')

five_percent_gdf = gpd.GeoDataFrame(five_percent, crs='EPSG:4326').set_index('iso_a3')

In [49]:
def embed_map(m, filename):
    from IPython.display import IFrame
    m.save(filename)
    return IFrame(filename, width='100%', height='700px')    

In [52]:
m = folium.Map(location=[0.0, 0.0], tiles='openstreetmap', zoom_start=3)

Choropleth(geo_data=five_percent_gdf.__geo_interface__,
          data=data['Percentage'],
          key_on='feature.id',
          legend_name='Countries with less than 5% healthcare expenditure',
          fill_color='GnBu', highlight=True, ).add_to(m)
embed_map(m, 'five_percent_healthcare.html')

In [54]:
five_percent.head()

,iso_a3,Percentage,geometry,Entity
0,AGO,3.730178,"MULTIPOLYGON (((12.99552 -4.78110, 12.63161 -4...",Angola
1,ARM,3.597931,"POLYGON ((46.50572 38.77061, 46.14362 38.74120...",Armenia
2,AZE,4.979777,"MULTIPOLYGON (((46.40495 41.86068, 46.68607 41...",Azerbaijan
3,BGD,3.166290,"POLYGON ((92.67272 22.04124, 92.65226 21.32405...",Bangladesh
4,BLR,4.754730,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",Belarus


In [53]:
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    five_percent,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Entity','Percentage'],
        aliases=['Country: ','Percentage: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)
folium.LayerControl().add_to(m)
m

ValueError: Cannot render objects with any missing geometries:    iso_a3  Percentage                                           geometry  \
0     AGO    3.730178  MULTIPOLYGON (((12.99552 -4.78110, 12.63161 -4...   
1     ARM    3.597931  POLYGON ((46.50572 38.77061, 46.14362 38.74120...   
2     AZE    4.979777  MULTIPOLYGON (((46.40495 41.86068, 46.68607 41...   
3     BGD    3.166290  POLYGON ((92.67272 22.04124, 92.65226 21.32405...   
4     BLR    4.754730  POLYGON ((28.17671 56.16913, 29.22951 55.91834...   
5     BEN    4.845360  POLYGON ((2.69170 6.25882, 1.86524 6.14216, 1....   
6     BTN    4.784856  POLYGON ((91.69666 27.77174, 92.10371 27.45261...   
7     BRN    1.996073  POLYGON ((115.45071 5.44773, 115.40570 4.95523...   
8     CMR    3.488587  POLYGON ((14.49579 12.85940, 14.89336 12.21905...   
9     CAF    3.687623  POLYGON ((27.37423 5.23394, 27.04407 5.12785, ...   
10    TCD    3.166192  POLYGON ((23.83766 19.58047, 23.88689 15.61084...   
11    CHN    4.960729  MU